## Deeplake

In [57]:
%pip install -r requirements.txt

  Using cached langchain-0.0.146-py3-none-any.whl (600 kB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.161
    Uninstalling langchain-0.0.161:
      Successfully uninstalled langchain-0.0.161
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index 0.5.25 requires langchain==0.0.142, but you have langchain 0.0.146 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [58]:
import os
from dotenv import load_dotenv
load_dotenv()

AZURE_DEPLOYMENT_NAME= os.environ['AZURE_DEPLOYMENT_NAME']
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_KEY"] = os.environ['AZURE_OPEN_API_KEY']
os.environ["OPENAI_API_BASE"] = os.environ['AZURE_API_BASE']
os.environ["OPENAI_VERSION"] = "2023-03-15-preview"   
os.environ["ACTIVELOOP_TOKEN"] = os.environ['ACTIVELOOP_TOKEN']  


In [59]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI 
from langchain.chains import RetrievalQA
from langchain.llms import OpenAIChat
from langchain.document_loaders import PagedPDFSplitter


embeddings = OpenAIEmbeddings(deployment="text-embedding-ada-002")
docsearch = Chroma.from_texts(texts, embeddings)



In [60]:
import requests
import tqdm
from typing import List

# https://www.digital.go.jp/assets/contents/node/basic_page/field_ref_resources/c5365b9c-0cf7-40b7-a5ff-b9f09e2d9292/615d206b/20221223_meeting_web3_summary_01.pdf
# https://www.digital.go.jp/assets/contents/node/basic_page/field_ref_resources/baf62741-ddf5-4c34-997c-9744a19cf8df/6fb98ecb/20221213_meeting_web3_summary_01.pdf

# financial reports of amamzon, but can be replaced by any URLs of pdfs
urls = ['https://www.digital.go.jp/assets/contents/node/basic_page/field_ref_resources/c5365b9c-0cf7-40b7-a5ff-b9f09e2d9292/615d206b/20221223_meeting_web3_summary_01.pdf',
        'https://www.digital.go.jp/assets/contents/node/basic_page/field_ref_resources/baf62741-ddf5-4c34-997c-9744a19cf8df/6fb98ecb/20221213_meeting_web3_summary_01.pdf'
        ]

def load_reports(urls: List[str]) -> List[str]:
    """ Load pages from a list of urls"""
    pages = []

    for url in tqdm.tqdm(urls):
        r = requests.get(url)
        path = url.split('/')[-1]
        with open(path, 'wb') as f:
            f.write(r.content)
        loader = PagedPDFSplitter(path)
        local_pages = loader.load_and_split()
        pages.extend(local_pages)
    return pages

pages = load_reports(urls)


100%|██████████| 2/2 [00:01<00:00,  1.23it/s]


In [61]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(pages)

In [62]:
embeddings = OpenAIEmbeddings(deployment="text-embedding-ada-002",chunk_size=1)

db = DeepLake(
    dataset_path="hub://tkc/dataset_name", 
    embedding_function=embeddings, 
    token=os.environ['ACTIVELOOP_TOKEN'])

db.add_documents(texts)


/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/tkc/dataset_name



|

hub://tkc/dataset_name loaded successfully.



Deep Lake Dataset in hub://tkc/dataset_name already exists, loading from the storage


Dataset(path='hub://tkc/dataset_name', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype      shape      dtype  compression
  -------   -------    -------    -------  ------- 
 embedding  generic  (108, 1536)  float32   None   
    ids      text     (108, 1)      str     None   
 metadata    json     (108, 1)      str     None   
   text      text     (108, 1)      str     None   


Evaluating ingest: 100%|██████████| 1/1 [00:37<00:00
\

Dataset(path='hub://tkc/dataset_name', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype      shape      dtype  compression
  -------   -------    -------    -------  ------- 
 embedding  generic  (130, 1536)  float32   None   
    ids      text     (130, 1)      str     None   
 metadata    json     (130, 1)      str     None   
   text      text     (130, 1)      str     None   


['2ef881a4-eca4-11ed-8ca9-1e00073ed291',
 '2ef8828a-eca4-11ed-8ca9-1e00073ed291',
 '2ef882b2-eca4-11ed-8ca9-1e00073ed291',
 '2ef882d0-eca4-11ed-8ca9-1e00073ed291',
 '2ef882ee-eca4-11ed-8ca9-1e00073ed291',
 '2ef88302-eca4-11ed-8ca9-1e00073ed291',
 '2ef88320-eca4-11ed-8ca9-1e00073ed291',
 '2ef8833e-eca4-11ed-8ca9-1e00073ed291',
 '2ef8835c-eca4-11ed-8ca9-1e00073ed291',
 '2ef8837a-eca4-11ed-8ca9-1e00073ed291',
 '2ef8838e-eca4-11ed-8ca9-1e00073ed291',
 '2ef883ac-eca4-11ed-8ca9-1e00073ed291',
 '2ef883ca-eca4-11ed-8ca9-1e00073ed291',
 '2ef883de-eca4-11ed-8ca9-1e00073ed291',
 '2ef883fc-eca4-11ed-8ca9-1e00073ed291',
 '2ef8841a-eca4-11ed-8ca9-1e00073ed291',
 '2ef8842e-eca4-11ed-8ca9-1e00073ed291',
 '2ef8844c-eca4-11ed-8ca9-1e00073ed291',
 '2ef8846a-eca4-11ed-8ca9-1e00073ed291',
 '2ef8847e-eca4-11ed-8ca9-1e00073ed291',
 '2ef8849c-eca4-11ed-8ca9-1e00073ed291',
 '2ef884ba-eca4-11ed-8ca9-1e00073ed291']

In [63]:
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    client=None,
    deployment_name=AZURE_DEPLOYMENT_NAME,
    openai_api_base=os.environ['AZURE_API_BASE'],
    openai_api_version="2023-03-15-preview",
    openai_api_key=os.environ['AZURE_OPEN_API_KEY'],
    temperature=0,
    request_timeout=180,
)


In [64]:
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type='stuff', 
    retriever=db.as_retriever())

qa.run("松尾")


"I'm sorry, I don't see a question here. Can you please provide more context or ask a specific question?"